### 000_TOC
First **Exercise:** how to do print `Hello, World!`?

In [1]:
echo "Hello, World!"
echo Hello, World!
printf %s Hello World!

Hello, World!
Hello, World!
HelloWorld!

### 010_functions
* write a function that takes three arguments (firstname, lastname, birthdate) and prints two lines: a welcome plus stating the birthday.
* write a function `mkcd` that takes an argument, creates a folder of that name and automatically `cd`s into it

In [2]:
welcome () {
    echo -en "Hi, $1 $2.\nYour birthday is on $3"
}
name=julian
welcome $name bla 31.12.2022

Hi, julian bla.
Your birthday is on 31.12.2022

In [3]:
mkcd () {
    mkdir -p $1; cd $1
}

# 020_pipes
some manipulations

In [4]:
bar () {
    echo stdin  >&1
    echo stderr >&2
}

discardStdin() {
    echo -n ""
}

echo "0.0"
bar
echo "0.1"
bar >/dev/null
echo "0.2"
bar 2>/dev/null

echo "1"
bar | discardStdin

echo "2"
bar |& discardStdin

echo "3"
(bar 3>&1 1>&2 2>&3) | discardStdin

0.0
stdin
stderr
0.1
stderr
0.2
stdin
1
stderr
2
3
stdin


### 030_subshells
**Exercise:** what is the number of files (including hidden files) times 3 minus the number of variables in env?

In [5]:
numberOfFiles=$(ls -A | wc -l)
ThreeTimesNumberOfFiles=$((numberOfFiles * 3))
echo $ThreeTimesNumberOfFiles

numberOfVariables=$(env | wc -l)
echo $(($ThreeTimesNumberOfFiles - $numberOfVariables))

57
7


In [6]:
echo $(($(ls -A | wc -l) * 3 - $(env | wc -l) ))


7


### 040_loops
* Return the numbers 5 to 10
* For each file in the folder, list the unix timestamp (seconds since Epoch) of the last modification (see `stat` for this)
* borrowed from https://missing.csail.mit.edu/2020/shell-tools/:
> Say you have a command that fails rarely. In order to debug it you need to capture its output but it can be time consuming to get a failure run. Write a bash script that runs the following script until it fails and captures its standard output and error streams to files and prints everything at the end. Bonus points if you can also report how many runs it took for the script to fail.

In [7]:
# seq 5 | awk '{print $1+5}'
for i in $(seq 5 10); do echo $i; done

5
6
7
8
9
10


In [8]:
# for f in $(ls); do stat --format %Y "$f"; done
for f in $(ls); do echo -e "${f}:\t $(stat -c %Y "$f")"; done

000_TOC.ipynb:	 1671646184
010_functions.ipynb:	 1671646411
020_pipes.ipynb:	 1671646434
030_subshells.ipynb:	 1671646470
040_loops.ipynb:	 1671646759
050_conditionals.ipynb:	 1671647301
060_regex.ipynb:	 1671647547
070_usefulCommands.ipynb:	 1670366793
090_advancedStuff.ipynb:	 1671624717
notebooks.tar:	 1670582036
playgroundAndSolutions.ipynb:	 1671647541
README.md:	 1669925935
temporaryfilethatwedeleteafterwards.txt:	 1670580620
tmpfile.log:	 1671646716
tmpscript_loops.sh:	 1671646694
tmpscript.sh:	 1670578503


In [9]:
crashOccured=false
for i in $(seq 100); do 
    bash tmpscript_loops.sh &> tmpfile.log
    if [ "$?" -ne 0 ]; then
        crashOccured=true
        break
    fi
done
if $crashOccured; then
    echo "error occured in iteration $i. Output was:"
    cat tmpfile.log
else
    echo "rerun to let error occur"
fi

error occured in iteration 84. Output was:
Something went wrong
The error was using magic numbers


### 050_conditionals
* Results of `-e`, `-d`, `-f` when we give a folder?
* Write a function that prints whether the argument is a file or a directory
* what does the script below do
```
filename=~/.tmp_squeueRepeat_new
timeforGeneration=20
timeSinceTouch=$(($(date +%s) - $(stat -c %Y $filename)))
redoAfter=$(($repeatEvery + $timeforGeneration))
if [ $timeSinceTouch -lt $redoAfter ]; then
[...]
```

In [10]:
[ -f ".git" ] && echo "true" || echo "false"
[ -d ".git" ] && echo "true" || echo "false"
[ -e ".git" ] && echo "true" || echo "false"

false
true
true


In [11]:
function whatisargument () {
    if [ "$#" -ne 1 ]; then
        echo "Function needs exactly one argument";
        return 1
    fi
    if ! [ -e "$1" ]; then
        echo "Argument must be an existing path"
        return 1
    fi
    
    if [ -f "$1" ]; then
        echo "argument '$1' is a file";
    elif [ -d "$1" ]; then
        echo "argument '$1' is a directory";
    else
        echo "argument '$1' is neither file nor directory but exists";
        return 1
    fi
}

whatisargument
whatisargument .git 050_conditionals.ipynb
whatisargument 050_conditionals.ipynb
whatisargument .git

Function needs exactly one argument
Function needs exactly one argument
argument '050_conditionals.ipynb' is a file
argument '.git' is a directory


The script checks how many seconds ago `$filename` was modified, and compares this agains the sum of two ints `$repeatEvery` and `$timeforGeneration`

### 060_regex
* Write a function `clearFilename` that get's a path and returns only the alphanumeric parts (bonus for doing it once with sed, and once with grep)
* Write a function that takes a filename 'foobarr_2.pickle' and returns it with the extension, the number and underscore removed. How would you automatically rename all/a subset of files in a folder using this function?
* Given an squeue function like below, return all jobs running on a RyzenHost with only the last 3 digits of their jobid and their username

In [12]:
function clearfilename () {
    # echo "$1" | grep -oP "[a-zA-Z0-9]*" | tr '\n' ' '
    echo "$1" | sed 's/[^a-zA-Z0-9]//g'
}
clearfilename foobarr_2.pickle

foobarr2pickle


In [13]:
squeue () {
    cat <<-EOF
Thu Dec 01 19:54:51 2022
             JOBID PARTITION     NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON) 
           4479803      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4486967      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4487128      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4487826      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4488503      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4492183      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4503427      cube clustern quiggeld  RUNNING    2:22:00   6:00:00      1 RyzenHost0 
           4503266      cube clustern quiggeld  RUNNING    4:11:31   6:00:00      1 RyzenHost0 
           4503251      cube clustern quiggeld  RUNNING    4:19:00   6:00:00      1 RyzenHost0 
           4503355  cubectrl     bash rheinema  RUNNING    3:03:15  12:00:00      1 HBPHost19 
           4459022      einc singular luboeins  RUNNING 21-05:45:01 UNLIMITED      1 EINCHost0 
           4460179      einc singular luboeins  RUNNING 20-01:38:19 UNLIMITED      1 EINCHost0 
           4502964    extoll     bash  thommes  RUNNING    7:00:00 UNLIMITED      1 EXTHost1 
EOF
}

squeue

Thu Dec 01 19:54:51 2022
             JOBID PARTITION     NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON) 
           4479803      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4486967      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4487128      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4487826      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4488503      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4492183      cube clustern quiggeld  PENDING       0:00   6:00:00      1 (launch failed requeued held) 
           4503427      cube clustern quiggeld  RUNNING    2:22:00   6:00:00      1 RyzenHost0 
           4503266      cube clustern quiggeld  RUNNING    4:11:31   6:00:00      1 Ryz

In [14]:
squeue | grep RyzenHost | sed -E 's/\s+/ /g' | cut -f 2,5  -d " " | grep -oP "[0-9]{3} [0-9a-zA-Z]*"

427 quiggeld
266 quiggeld
251 quiggeld


In [15]:
squeue | grep RyzenHost | sed -E 's#^\s*[0-9]*([0-9]{3})(\s*\S*){2}\s*(\S*).*#job by \3 and shortened jobid \1#g'

job by quiggeld and shortened jobid 427
job by quiggeld and shortened jobid 266
job by quiggeld and shortened jobid 251


In [16]:
squeue | while read line; do
    # only process if RyzenHost is there
    echo $line | grep -q RyzenHost || continue
    # using cut:
    info_user="$(echo $line | cut -f 4 -d ' ')"
    info_jobid="$(echo $line | cut -f 1 -d ' ')"
    # using grep
    info_jobid="$(echo $line | grep -oP '^(\s*\S*){0}\s*\K\S*')"
    info_user="$(echo $line | grep -oP '^(\s*\S*){3}\s*\K\S*')"
    # for playing around:
    # echo $line | grep -P --color=auto '^(\s*\S*){0}\s*\K\S*'
    
    # getting the last three digits
    info_jobidLastThree="$(echo $info_jobid | grep -oP "[0-9]{3}$")"
    
    
    echo "job by $info_user and shortened jobid $info_jobidLastThree"
done

job by quiggeld and shortened jobid 427
job by quiggeld and shortened jobid 266
job by quiggeld and shortened jobid 251
